In [165]:
import re
import pandas as pd

In [166]:
# read csv files list_of_jobs_jobstreet.csv
job_dataset = pd.read_csv('list_of_jobs_jobstreet.csv')

In [167]:
job_dataset.isnull().sum()

Job Title                     0
Company Name               1199
Location                      0
Salary                    24464
Job Classification            0
Job Sub Classification        0
Job Description           31998
Facility                  25873
Posted Date                   0
More Detail                   0
dtype: int64

In [168]:
# drop Job Description collumn
job_dataset = job_dataset.drop(['Job Description'], axis=1)
# replace null value
job_dataset['Company Name'].fillna('anonymous', inplace=True)
job_dataset['Salary'].fillna('not_written', inplace=True)
job_dataset['Facility'].fillna('not_written', inplace=True)

In [169]:
job_dataset.drop_duplicates(inplace=True)
job_dataset.reset_index(drop=True, inplace=True)
job_dataset.shape

(25034, 9)

In [170]:
def clean_column(job_dataset, column_name):
    corpus = []
    for i in range(0, len(job_dataset)):
        text = re.sub('[^a-zA-Z]', ' ', job_dataset[column_name][i])
        text = text.lower()
        text = text.split()
        text = ' '.join(text)
        corpus.append(text)

    job_dataset[column_name] = corpus
    return job_dataset

In [171]:
job_dataset = clean_column(job_dataset, 'Job Title')
job_dataset = clean_column(job_dataset, 'Company Name')

In [172]:
job_dataset = clean_column(job_dataset, 'Job Classification')
job_dataset = clean_column(job_dataset, 'Job Sub Classification')
job_dataset = clean_column(job_dataset, 'Location')

In [173]:
def checking_salary_period(salary):
    if 'year' in salary:
        return 'year'
    elif 'month' in salary:
        return 'month'
    elif 'week' in salary:
        return 'week'
    elif 'day' in salary:
        return 'day'
    elif 'hour' in salary:
        return 'hour'
    else:
        return 'not_written'

In [174]:
job_dataset['og_period_salary'] = job_dataset['Salary'].apply(checking_salary_period)

In [175]:
def cleaning_salary_period(salary):
    if 'per month' in salary:
        salary = re.sub(r'per month', '', salary)
    elif 'per year' in salary:
        salary = re.sub(r'per year', '', salary)
    elif 'per week' in salary:
        salary = re.sub(r'per week', '', salary)
    elif 'per day' in salary:
        salary = re.sub(r'per day', '', salary)
    elif 'per hour' in salary:
        salary = re.sub(r'per hour', '', salary)
    return salary

In [176]:
job_dataset['Salary'] = job_dataset['Salary'].apply(cleaning_salary_period)

In [177]:
# job_dataset[(job_dataset['Salary'] != 'not_written') & ~(job_dataset['Salary'].str.contains('[0-9]')) & (job_dataset['Salary'].str.contains('-'))].replace('-','not_written')

In [178]:
# job_dataset[(job_dataset['Salary'] != 'not_written') & ~(job_dataset['Salary'].str.contains('-')) & ~(job_dataset['Salary'].str.contains('–')) & ~(job_dataset['Salary'].str.contains('Rp')) & ~(job_dataset['Salary'].str.contains('[0-9]')) & (job_dataset['Salary'] == 'TBC')].replace('TBC','not_written')

In [179]:
job_dataset['Compensation'] = job_dataset[(job_dataset['Salary'] != 'not_written') & ~(job_dataset['Salary'].str.contains('-')) & ~(job_dataset['Salary'].str.contains('–')) & ~(job_dataset['Salary'].str.contains('[0-9]'))]['Salary']

In [180]:
def checking_currency(salary):
    if 'Rp' in salary:
        return 'IDR'
    elif 'IDR' in salary:
        return 'IDR'
    elif 'USD' in salary:
        return 'USD'
    elif '$' in salary:
        return 'USD'
    elif 'RM' in salary:
        return 'RM'
    elif 'SGD' in salary:
        return 'SGD'
    elif 'MYR' in salary:
        return 'MYR'
    elif 'AUD' in salary:
        return 'AUD'
    elif 'EUR' in salary:
        return 'EUR'
    elif 'HKD' in salary:
        return 'HKD'
    elif '฿' in salary:
        return 'THB'
    elif 'THB' in salary:
        return 'THB'
    else:
        return 'not_written'

In [181]:
job_dataset['og_salary_currency'] = job_dataset['Salary'].apply(checking_currency)

In [182]:
def cleaning_currency(salary):
    currency_symbols = ['IDR', 'Rp', '$', '(USD)', 'SGD', 'MYR', 'AUD', 'EUR', 'RM', 'HKD', '฿', '(THB)', '()']
    
    for symbol in currency_symbols:
        salary = salary.replace(symbol, '')

    if ' ' in salary:
        salary = salary.replace(' ', '')
    elif salary == 'not_written':
        salary = salary

    return salary

In [183]:
job_dataset['Salary'] = job_dataset['Salary'].apply(cleaning_currency)

In [184]:
job_dataset['Salary'] = job_dataset['Salary'].str.replace(',', '')
job_dataset['Salary'] = job_dataset['Salary'].str.replace('.', '')
# remove blank space
job_dataset['Salary'] = job_dataset['Salary'].str.strip()
job_dataset['Salary'] = job_dataset['Salary'].str.replace(' ', '')


In [185]:
# replace Nan value in compensation with 'not_written'
job_dataset['Compensation'].fillna('not_written', inplace=True)

In [186]:
job_dataset['Salary']
job_dataset = job_dataset[job_dataset['Salary'] != 'not_written']
job_dataset['Salary'].to_csv('job_salary.csv', index=False)

In [187]:
def average_salary(row):
    try:
        salary = row['Salary']
        if '-' in salary:
            salary = salary.split('-')
            salary = list(map(int, salary))
            salary = sum(salary)/len(salary)
        else:
            salary = salary
        return salary
    except ValueError:
        print(f"Invalid value for salary: {salary} in row {row}")

In [188]:
job_dataset['Salary'] = job_dataset.apply(average_salary, axis=1)

In [189]:
def extract_job_id(url):
    parts = url.split('/')
    job_id_with_query = parts[-1]
    job_id = job_id_with_query.split('?')[0]
    return job_id

In [190]:
job_dataset['Job ID'] = job_dataset['More Detail'].apply(extract_job_id)

In [191]:
job_dataset = job_dataset.drop_duplicates(subset='Job ID', keep='first')

In [193]:
job_dataset.to_csv('jobstreet_cleaned.csv', index=False)